Um padrão de Agent Otimizador de Prompt realiza Chain-of-Thought (CoT) com reflexão e ação no ambiente. Vamos estruturar o agente para seguir o fluxo de geração de resultado, execução de ação e reflexão.


### Explicação do Código

1. **LLM**: Utilizamos a API do OpenAI (GPT-3 ou GPT-4) para responder às perguntas com base em **Chain of Thought (CoT)**. Esse método faz com que o modelo raciocine passo a passo para chegar à solução.

2. **Prompt Template**: O template para CoT orienta o LLM a resolver o problema com uma abordagem mais estruturada e detalhada.

3. **Agente**: A função `agent` segue o padrão da imagem:
   - **Geração do Resultado**: O agente gera um resultado inicial baseado no prompt do problema.
   - **Reflexão**: O agente recebe feedback do ambiente e reflete sobre a ação anterior.
   - **Otimização**: Com base na reflexão, o agente pode otimizar sua resposta e gerar um novo resultado.

4. **Ambiente**: Simula o feedback que o agente recebe após a execução da ação. No exemplo, ele valida se a resposta está correta ou incorreta.

### Execução

Quando você executa esse script, o agente vai:
1. Resolver o problema com raciocínio CoT.
2. Refletir com base no feedback do ambiente.
3. Gerar uma nova resposta se necessário, após a reflexão.

Este código segue o padrão de agente da imagem, onde há um ciclo de **ação**, **reflexão**, e **interação com o ambiente**.





In [ ]:
pip install langchain openai

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
import os

# Definir sua chave de API da OpenAI (você precisa ter uma conta e uma chave)
os.environ["OPENAI_API_KEY"] = "sua-chave-api-openai-aqui"

# Definir o modelo LLM (você pode mudar para GPT-4 se tiver acesso)
llm = OpenAI(model="text-davinci-003")

# Template de Prompt para usar Chain-of-Thought (CoT)
cot_template = """
Use chain of thought reasoning to solve the following problem:
Input: {problem}
"""

# Criação do Prompt
prompt = PromptTemplate(
    input_variables=["problem"],
    template=cot_template
)

# Função que cria uma reflexão para o agente (similar ao que está na imagem)
def agent_reflection(result, environment_feedback):
    reflection_prompt = f"""
    Reflect on the result of your previous action:
    Result: {result}
    Environment Feedback: {environment_feedback}

    What would you improve in your next action?
    """
    reflection_chain = LLMChain(llm=llm, prompt=PromptTemplate(template=reflection_prompt, input_variables=[]))
    reflection = reflection_chain.run({})
    return reflection

# Função principal do agente
def agent(problem, environment):
    # 1. Geração de resultado inicial (CoT reasoning)
    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run(problem)

    print("Initial Result: ", result)

    # 2. Refletir sobre o resultado, pegando feedback do ambiente
    feedback = environment.get_feedback(result)
    reflection = agent_reflection(result, feedback)

    print("Agent Reflection: ", reflection)

    # 3. Gerar nova ação baseada na reflexão
    final_result = chain.run(problem)

    print("Final Result after reflection: ", final_result)
    return final_result

# Simulação do ambiente para feedback
class Environment:
    def get_feedback(self, result):
        # Aqui poderia estar uma lógica mais complexa de feedback
        if "230 miles" in result:
            return "The answer seems correct."
        else:
            return "The answer may be incorrect. Try again."

# Testar o agente com um problema
if __name__ == "__main__":
    problem = "Matteo traveled at 55 miles per hour for 4 hours. Shandy traveled at 45 miles per hour for 10 hours. How many miles farther did Shandy drive than Matteo?"

    # Criar ambiente para dar feedback ao agente
    environment = Environment()

    # Executar o agente com o problema
    agent(problem, environment)
